## Multiple Chain 快速入门

Runnables 可以轻松地用来串联多个 Chains，使用 RunnablePassthrough 将输出同时传给多条后继链。

```
     Input
      / \
     /   \
 Chain1 Chain2
     \   /
      \ /
      Combine
```

本指南展示如何使用 Runnable 实现多个 AI 关于相同话题的辩论：

```
    输入话题
       |
       |
    原始观点
      / \
     /   \
 正面论述 反面论述
     \   /
      \ /
     最终总结
```

In [4]:
# 导入相关模块，包括运算符、输出解析器、聊天模板、ChatOpenAI 和 运行器
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough

# 创建一个计划器，生成一个关于给定输入的论证
planner = (
    ChatPromptTemplate.from_template("生成关于以下内容的论点: {input}")
    | ChatOpenAI(model="gpt-4o-mini")
    | StrOutputParser()
    | {"base_response": RunnablePassthrough()}
)

# 创建正面论证的处理链，列出关于基础回应的正面或有利的方面
arguments_for = (
    ChatPromptTemplate.from_template(
        "列出关于{base_response}的正面或有利的方面"
    )
    | ChatOpenAI(model="gpt-4o-mini")
    | StrOutputParser()
)

# 创建反面论证的处理链，列出关于基础回应的反面或不利的方面
arguments_against = (
    ChatPromptTemplate.from_template(
        "列出关于{base_response}的反面或不利的方面"
    )
    | ChatOpenAI(model="gpt-4o-mini")
    | StrOutputParser()
)

# 创建最终响应者，综合原始回应和正反论点生成最终的回应
final_responder = (
    ChatPromptTemplate.from_messages(
        [
            ("ai", "{original_response}"),
            ("human", "正面观点:\n{results_1}\n\n反面观点:\n{results_2}"),
            ("system", "给出批评后生成最终回应"),
        ]
    )
    | ChatOpenAI(model="gpt-4o-mini")
    | StrOutputParser()
)

# 构建完整的处理链，从生成论点到列出正反论点，再到生成最终回应
chain = (
    planner
    | {
        "results_1": arguments_for,
        "results_2": arguments_against,
        "original_response": itemgetter("base_response"),
    }
    | final_responder
)

In [5]:
chain.invoke({"input": "中国经济"})

'针对上述关于中国经济的正面和反面观点，我们可以进行综合分析，以更全面地理解中国经济的现状和未来发展潜力。\n\n### 正面观点的回应\n\n1. **持续的经济增长**：尽管增速有所放缓，但中国经济依然在全球经济中发挥着重要作用。政府的政策调整和市场机制的完善，有助于维持经济的相对稳定。\n\n2. **强大的制造业基础**：尽管面临竞争压力，中国制造业的升级转型正在进行。智能制造和技术创新将帮助提升制造业的附加值，增强全球竞争力。\n\n3. **科技创新的加速**：中国在科技领域的持续投资和政策支持，正在为经济注入新的活力，尤其是在数字经济和绿色技术方面。\n\n4. **庞大的内需市场**：随着消费结构的逐步升级，内需市场的潜力仍然巨大。政府的刺激政策将进一步推动消费增长。\n\n5. **国际贸易的多元化**：一带一路倡议的实施，尽管面临风险，但也为中国企业开辟了新的市场，增强了经济抗风险能力。\n\n6. **环境保护与绿色经济的重视**：政府在推动经济发展的同时，逐步加强环境保护的力度，力求实现经济增长与生态环境的协调。\n\n7. **数字经济的快速崛起**：数字经济的迅猛发展为传统产业转型提供了机会，提升了整体经济效率。\n\n8. **政策灵活性与政府支持**：中国政府的宏观调控能力使得经济能够及时应对内外部挑战，为经济的稳定发展提供了保障。\n\n9. **社会基础设施的不断改善**：基础设施建设的持续推进为经济发展提供了必要的支持，促进了区域经济的一体化发展。\n\n### 反面观点的回应\n\n1. **经济增长放缓**：虽然增速放缓，但这也反映了经济结构的转型过程，长期来看，有助于实现更高质量的发展。\n\n2. **制造业过剩和竞争压力**：面对竞争压力，中国制造业正在转型升级，通过技术创新和高附加值产品提升竞争力。\n\n3. **内需不足与消费结构问题**：政府的消费刺激和政策支持正在逐步改善消费信心，推动消费结构向服务业和高品质商品转型。\n\n4. **一带一路风险**：虽然面临风险，但通过加强与参与国的合作与沟通，中国可以更好地管理项目风险，实现共赢发展。\n\n5. **环境治理难题**：中国政府意识到环境保护的重要性，正在实施更为严格的环保政策，努力解决环境问题。\n\n6. **人口老龄化加剧的经济负担**：政府正在积

#### 流式输出

In [6]:
## chain 最终输出经过了 StrOutputParser 处理，所以可以直接输出流式输出 s
for s in chain.stream({"input": "中国经济"}):
    print(s, end="", flush=True)

在对中国经济的正面与反面观点进行深入分析后，我们可以得出一个更全面的结论，既考虑到其潜力，也关注到面临的挑战。以下是对上述观点的最终回应：

### 综合回应

中国经济在过去几十年中取得了显著的成就，展现出强大的增长潜力和韧性。其庞大的市场规模、不断扩大的中产阶级以及科技创新的加速发展，为未来的经济增长奠定了坚实基础。同时，中国在全球供应链中的关键角色和基础设施建设的持续投资，使其在国际经济中保持了竞争力。然而，这些正面因素并不能掩盖中国经济面临的一系列挑战。

首先，经济增长速度的放缓确实引发了对就业和社会稳定的担忧，尤其是在全球经济形势不确定的背景下。中国需要找到新的增长动力，可能需要通过进一步的产业升级和消费驱动来实现。其次，尽管科技投资不断增加，但在某些核心技术上仍依赖外部，这表明中国在自主创新能力方面还有提升的空间。

此外，贸易摩擦和国际关系的紧张可能影响中国的出口导向型经济，带来不确定性，企业需要在这样的环境中调整策略。环境污染和资源枯竭的问题同样不容忽视，这是一个长期且复杂的挑战，需要政府和社会共同努力。

人口老龄化和区域发展失衡的问题，则是中国在发展过程中必须面对的结构性挑战。如何有效应对这些问题，将直接影响到社会的可持续发展与经济的长期稳定。

最后，在政策干预与市场机制的平衡方面，过度的政策干预可能抑制市场活力，因此政府需要在促进经济发展的同时，保持市场的健康竞争。

### 结论

在展望未来时，中国经济既有希望也面临挑战，关键在于如何利用已有的优势，同时有效应对这些挑战。通过进一步深化改革，推动科技创新，优化资源配置，中国有潜力实现更加可持续和包容的经济增长。

### Homework: 实现一个多链版本的代码生成，输入功能需求，输出2种（Python，Java）以上编程语言的代码实现。

In [10]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough

# 初始化 OpenAI 模型
model = ChatOpenAI(model="gpt-4o-mini")

# 创建代码生成模板
code_gen_template = """
You are an expert programmer. Write a {language} implementation of the following:

{requirement}

Only provide the code, no explanations. Make sure the code is complete and can be run directly.
"""

# 为每种语言创建一个链
def create_language_chain(language):
    prompt = ChatPromptTemplate.from_template(code_gen_template)
    return (
        {"language": lambda _: language, "requirement": RunnablePassthrough()}
        | prompt
        | model
        | StrOutputParser()
    )

# 创建并行运行的多链
multi_language_chain = RunnableParallel(
    python=create_language_chain("Python"),
    java=create_language_chain("Java"),
    csharp=create_language_chain("C#")
)

# 运行多链并获取结果
requirement = "Write a quicksort algorithm"
results = multi_language_chain.invoke(requirement)

# 打印结果
for language, code in results.items():
    print(f"\n--- {language.upper()} Implementation ---")
    print(code)


--- PYTHON Implementation ---
```python
def quicksort(arr):
    if len(arr) <= 1:
        return arr
    pivot = arr[len(arr) // 2]
    left = [x for x in arr if x < pivot]
    middle = [x for x in arr if x == pivot]
    right = [x for x in arr if x > pivot]
    return quicksort(left) + middle + quicksort(right)

if __name__ == "__main__":
    array = [3, 6, 8, 10, 1, 2, 1]
    sorted_array = quicksort(array)
    print("Sorted array:", sorted_array)
```

--- JAVA Implementation ---
```java
import java.util.Arrays;

public class QuickSort {
    public static void main(String[] args) {
        int[] array = {3, 6, 8, 10, 1, 2, 1};
        quickSort(array, 0, array.length - 1);
        System.out.println("Sorted array: " + Arrays.toString(array));
    }

    public static void quickSort(int[] array, int low, int high) {
        if (low < high) {
            int partitionIndex = partition(array, low, high);
            quickSort(array, low, partitionIndex - 1);
            quickSort(array

In [9]:
def quicksort(arr):
    if len(arr) <= 1:
        return arr
    pivot = arr[len(arr) // 2]
    left = [x for x in arr if x < pivot]
    middle = [x for x in arr if x == pivot]
    right = [x for x in arr if x > pivot]
    return quicksort(left) + middle + quicksort(right)

if __name__ == "__main__":
    array = [ 1, 3, 3, 2]
    sorted_array = quicksort(array)
    print("Sorted array:", sorted_array)

Sorted array: [1, 2, 3, 3]
